In [1]:
import os
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['font.size'] = 12
plt.ioff()
import h5py
import pandas as pd
from tqdm.notebook import tqdm
from skimage.measure import block_reduce
import scipy.ndimage
from PIL import Image


In [2]:
f = h5py.File("data_files/nue_WithWire_00.h5")

wire_table_rse_num_tups = np.array(f['wire_table']['event_id'], dtype=np.uint64)
particle_table_rse_num_tups = np.array(f['particle_table']['event_id'], dtype=np.uint64)

wire_table_rse_nums = wire_table_rse_num_tups[:, 0] * 1_000_000_000_000 + wire_table_rse_num_tups[:, 1] * 1_000_000 + wire_table_rse_num_tups[:, 2]
particle_table_rse_nums = particle_table_rse_num_tups[:, 0] * 1_000_000_000_000 + particle_table_rse_num_tups[:, 1] * 1_000_000 + particle_table_rse_num_tups[:, 2]

unique_rse_nums = np.unique(wire_table_rse_nums)


In [3]:
all_adcs = f['wire_table']['adc']
all_local_wires = f['wire_table']['local_wire']
all_local_planes = f['wire_table']['local_plane']

particle_start_wire_pos = f['particle_table']['start_wire_pos']
particle_start_wire_time = f['particle_table']['start_wire_time']

os.system("rm event_display_images_hires/*.png")
os.system("rm event_display_images_lores/*.png")

extra_plot_debug = True

num_events = 100
num_gif_events = 20
image_stack = np.zeros((num_events, 128, 128), dtype=np.float32)
hires_image_list = []
for event_i in tqdm(range(num_events)):

    curr_rse_num = unique_rse_nums[event_i]
    curr_run_num = int(curr_rse_num // 1_000_000_000_000)
    curr_subrun_num = int((curr_rse_num % 1_000_000_000_000) // 1_000_000)
    curr_event_num = int(curr_rse_num % 1_000_000)

    # Only looking at first particle to get neutrino vertex info, for some reason the vertex from the event_table tree didn't seem to match.
    event_particle_start_wire_pos = np.array(particle_start_wire_pos[particle_table_rse_nums == unique_rse_nums[event_i]])[0]
    event_particle_start_wire_time = np.array(particle_start_wire_time[particle_table_rse_nums == unique_rse_nums[event_i]])[0]

    event_adcs = np.array(all_adcs[wire_table_rse_nums == unique_rse_nums[event_i]])
    event_local_wire_nums = np.array(all_local_wires[wire_table_rse_nums == unique_rse_nums[event_i]]).flatten()
    event_local_plane_nums = np.array(all_local_planes[wire_table_rse_nums == unique_rse_nums[event_i]]).flatten()

    collection_adcs = event_adcs[event_local_plane_nums == 2]
    collection_local_wire_nums = event_local_wire_nums[event_local_plane_nums == 2]
    collection_particle_start_wire_pos = event_particle_start_wire_pos[2]
    collection_particle_start_wire_time = event_particle_start_wire_time[0]

    wire_pitch = 0.3 # cm, from https://microboone.fnal.gov/wp-content/uploads/MICROBOONE-NOTE-1083-PUB.pdf
    time_tick = 0.5 # microseconds, from https://microboone.fnal.gov/wp-content/uploads/MICROBOONE-NOTE-1083-PUB.pdf
    drift_velocity = 0.114 # cm / microsecond # 114 cm / millisecond, from https://microboone.fnal.gov/wp-content/uploads/MICROBOONE-NOTE-1026-PUB.pdf

    nu_vtx_z = collection_particle_start_wire_pos * wire_pitch # measuring from leftmost wire, not exactly the same as detector coordinates!
    nu_vtx_x = collection_particle_start_wire_time * time_tick * drift_velocity # approximate and relative, not exactly the same as detector coordinates!

    f_downsample = 6
    collection_adcs = block_reduce(collection_adcs, block_size=(1, f_downsample), func=np.sum)

    adccutoff = 10.*f_downsample/6.
    adcsaturation = 100.*f_downsample/6.
    collection_adcs[collection_adcs < adccutoff] = 0
    collection_adcs[collection_adcs > adcsaturation] = adcsaturation
    # this is the standard form of ADCs, from the public data reference notebook

    def get_image_arr(resolution, size, extra_plot_debug=False):
        # resolution is pixels per side of image. resolution=None will use the maximum resolution, one pixel per wire.
        # size is cm per side of image

        if resolution: # manual resolution choice, for downsampling

            scaling_z = wire_pitch / size * resolution
            scaling_x = time_tick * f_downsample * drift_velocity / size * resolution
            img = Image.fromarray(np.array(collection_adcs.T * 255 / 100, dtype=np.uint8), mode="L")
            new_size = (int(img.width * scaling_z), int(img.height * scaling_x))
            scaled_arr = np.array(img.resize(new_size, Image.BOX)) * 100. / 255.

            scaling_z = wire_pitch / size * resolution
            scaling_x = time_tick * f_downsample * drift_velocity / size * resolution
            img = Image.fromarray(np.array(collection_adcs.T * 255 / 100, dtype=np.uint8), mode="L")
            new_size = (int(img.width * scaling_z), int(img.height * scaling_x))
            scaled_arr = np.array(img.resize(new_size, Image.BOX)) * 100. / 255.

            nu_vtx_index_z = int(collection_particle_start_wire_pos * scaling_z)
            nu_vtx_index_x = int(collection_particle_start_wire_time / f_downsample * scaling_x)

            num_pad_pixels = resolution
            padded_scaled_arr = np.pad(scaled_arr, ((num_pad_pixels, num_pad_pixels), (num_pad_pixels, num_pad_pixels)), mode='constant', constant_values=0)
            nu_vtx_index_z += num_pad_pixels
            nu_vtx_index_x += num_pad_pixels

            image_arr = padded_scaled_arr[nu_vtx_index_x - resolution // 2 : nu_vtx_index_x + resolution // 2, 
                                        nu_vtx_index_z - resolution // 2 : nu_vtx_index_z + resolution // 2]

        elif resolution == None: # keeping max resolution available, just fixing aspect ratio

            auto_resolution = size / wire_pitch
            scaling_z = 1
            scaling_x = time_tick * f_downsample * drift_velocity / size * auto_resolution
            
            img = Image.fromarray(np.array(collection_adcs.T * 255 / 100, dtype=np.uint8), mode="L")
            new_size = (int(img.width * scaling_z), int(img.height * scaling_x))
            scaled_arr = np.array(img.resize(new_size)) * 100. / 255.

            nu_vtx_index_z = int(collection_particle_start_wire_pos * scaling_z)
            nu_vtx_index_x = int(collection_particle_start_wire_time / f_downsample * scaling_x)

            num_pad_pixels = int(auto_resolution)
            padded_scaled_arr = np.pad(scaled_arr, ((num_pad_pixels, num_pad_pixels), (num_pad_pixels, num_pad_pixels)), mode='constant', constant_values=0)
            nu_vtx_index_z += num_pad_pixels
            nu_vtx_index_x += num_pad_pixels

            half_auto_resolution = int(auto_resolution // 2)
            image_arr = padded_scaled_arr[nu_vtx_index_x - half_auto_resolution : nu_vtx_index_x + half_auto_resolution, 
                                        nu_vtx_index_z - half_auto_resolution : nu_vtx_index_z + half_auto_resolution]


        return image_arr

    image_arr = get_image_arr(128, 200, extra_plot_debug=False)
    image_stack[event_i] = image_arr

    if event_i < num_gif_events:
        image_arr = get_image_arr(None, 200, extra_plot_debug=False)
        hires_image_list.append(image_arr)



  0%|          | 0/100 [00:00<?, ?it/s]

In [4]:
# saving some images, just for visualization purposes

for event_i in range(num_gif_events):

    image_array = image_stack[event_i]
    image_arr_normalized = image_array / 100.
    rgb_arr = plt.get_cmap('jet')(image_arr_normalized)[:, :, :3]
    img = Image.fromarray((rgb_arr * 255).astype(np.uint8))
    img.save(f"event_display_images_lores/event_{event_i:06d}.png")

    image_array = hires_image_list[event_i]
    image_arr_normalized = image_array / 100.
    rgb_arr = plt.get_cmap('jet')(image_arr_normalized)[:, :, :3]
    img = Image.fromarray((rgb_arr * 255).astype(np.uint8))
    img.save(f"event_display_images_hires/event_{event_i:06d}.png")
    

In [5]:
# making gifs, just for visualization purposes

images = []
for event_i in range(5):
    images.append(imageio.imread(f"event_display_images_hires/event_{event_i:06d}.png"))
imageio.mimsave('event_displays_hires.gif', images, loop=0, duration=100)

images = []
for event_i in range(5):
    images.append(imageio.imread(f"event_display_images_lores/event_{event_i:06d}.png"))
imageio.mimsave('event_displays_lores.gif', images, loop=0, duration=100)


/tmp/ipykernel_56914/4243937373.py:5: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f"event_display_images_hires/event_{event_i:06d}.png"))
/tmp/ipykernel_56914/4243937373.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f"event_display_images_lores/event_{event_i:06d}.png"))


In [6]:
np.savez("processed_data_files/nue_images.npz", image_stack=image_stack)
